# Import Dependencies

In [341]:
# Import and Dependencies
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import io
import os
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
ohe = OneHotEncoder()
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
# Colab Featured Interactive Table
from google.colab import data_table 
data_table.enable_dataframe_formatter()

# Read the CSV and Perform Basic Data Cleaning

In [342]:
# File Import from PC, CSV Reader
from google.colab import files
uploaded = files.upload()
lung_cancer_df = pd.read_csv('survey_lung_cancer.csv')

Saving survey_lung_cancer.csv to survey_lung_cancer (27).csv


In [343]:
# Examine DataFrame
lung_cancer_df

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2,YES
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2,YES
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2,NO
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2,NO
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,F,56,1,1,1,2,2,2,1,1,2,2,2,2,1,YES
305,M,70,2,1,1,1,1,2,2,2,2,2,2,1,2,YES
306,M,58,2,1,1,1,1,1,2,2,2,2,1,1,2,YES
307,M,67,2,1,2,1,1,2,2,1,2,2,2,1,2,YES


In [344]:
lung_cancer_df.columns

Index(['GENDER', 'AGE', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY',
       'PEER_PRESSURE', 'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING',
       'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH',
       'SWALLOWING DIFFICULTY', 'CHEST PAIN', 'LUNG_CANCER'],
      dtype='object')

In [345]:
# Drop unnecessary columns
lung_cancer_df=lung_cancer_df.drop(['PEER_PRESSURE', 'ALCOHOL CONSUMING', 'ALLERGY '],axis=1)

# Change Yes and No to 1 and 0
key_rev = {'YES' : 1, 'NO' : 0}
lung_cancer_df = lung_cancer_df.replace(key_rev)

# Change Male to 1 and Female to 2
key_rev_gender = {'M' : 1, 'F' : 2}
lung_cancer_df = lung_cancer_df.replace(key_rev_gender)

lung_cancer_df

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,CHRONIC DISEASE,FATIGUE,WHEEZING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN,LUNG_CANCER
0,1,69,1,2,2,1,2,2,2,2,2,2,1
1,1,74,2,1,1,2,2,1,1,2,2,2,1
2,2,59,1,1,1,1,2,2,2,2,1,2,0
3,1,63,2,2,2,1,1,1,1,1,2,2,0
4,2,63,1,2,1,1,1,2,2,2,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,2,56,1,1,1,2,2,1,2,2,2,1,1
305,1,70,2,1,1,1,2,2,2,2,1,2,1
306,1,58,2,1,1,1,1,2,2,1,1,2,1
307,1,67,2,1,2,1,2,1,2,2,1,2,1


# Split the Data into Training and Testing

In [346]:
# Create our Feature
X = lung_cancer_df.drop("LUNG_CANCER", axis=1)
X = pd.get_dummies(X)

# Create our Target
y = lung_cancer_df["LUNG_CANCER"]

In [347]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(f'Train set lenght: {X_train.count()} records')
print(f'Test set lenght: {X_test.count()} records')

Train set lenght: GENDER                   207
AGE                      207
SMOKING                  207
YELLOW_FINGERS           207
ANXIETY                  207
CHRONIC DISEASE          207
FATIGUE                  207
WHEEZING                 207
COUGHING                 207
SHORTNESS OF BREATH      207
SWALLOWING DIFFICULTY    207
CHEST PAIN               207
dtype: int64 records
Test set lenght: GENDER                   102
AGE                      102
SMOKING                  102
YELLOW_FINGERS           102
ANXIETY                  102
CHRONIC DISEASE          102
FATIGUE                  102
WHEEZING                 102
COUGHING                 102
SHORTNESS OF BREATH      102
SWALLOWING DIFFICULTY    102
CHEST PAIN               102
dtype: int64 records


In [348]:
# Check the Feature values
X.describe()

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,CHRONIC DISEASE,FATIGUE,WHEEZING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN
count,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000,309.000000
mean,1.475728,62.673139,1.563107,1.569579,1.498382,1.504854,1.673139,1.556634,1.579288,1.640777,1.469256,1.556634
std,0.500221,8.210301,0.496806,0.495938,0.500808,0.500787,0.469827,0.497588,0.494474,0.480551,0.499863,0.497588
min,1.000000,21.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,57.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,62.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000
75%,2.000000,69.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,2.000000,87.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [349]:
# Check teh balance of the Target values
y.value_counts()

1    270
0     39
Name: LUNG_CANCER, dtype: int64

In [350]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Supervised Machine Learning

### Naive Random Oversampling

In [351]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 197, 1: 197})

In [352]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=42)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=42)

In [353]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8246575342465754

In [354]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 4,  1],
       [11, 62]])

In [355]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.27      0.80      0.85      0.40      0.82      0.68         5
          1       0.98      0.85      0.80      0.91      0.82      0.68        73

avg / total       0.94      0.85      0.80      0.88      0.82      0.68        78



### SMOTE Oversampling

In [356]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_sampled, y_resampled = SMOTE(random_state=1,
                              sampling_strategy='auto').fit_resample(
X_train, y_train)

In [357]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=42)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=42)

In [358]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8246575342465754

In [359]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 4,  1],
       [11, 62]])

In [360]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.27      0.80      0.85      0.40      0.82      0.68         5
          1       0.98      0.85      0.80      0.91      0.82      0.68        73

avg / total       0.94      0.85      0.80      0.88      0.82      0.68        78



### Undersampling (ClusterCentroids)

In [361]:
# Resample the data using the ClusterCentroids resampler
from collections import Counter
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 34, 1: 34})

In [362]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=42)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=42)

In [363]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.8246575342465754

In [364]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 5,  0],
       [65,  8]])

In [365]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.07      1.00      0.11      0.13      0.33      0.12         5
          1       1.00      0.11      1.00      0.20      0.33      0.10        73

avg / total       0.94      0.17      0.94      0.19      0.33      0.10        78



### Combination Oversampling/Undersampling (SMOTEEN)

In [366]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({0: 238, 1: 209})

In [367]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=42)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=42)

In [368]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.5547945205479452

In [369]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 3,  2],
       [ 6, 67]])

In [370]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.33      0.60      0.92      0.43      0.74      0.53         5
          1       0.97      0.92      0.60      0.94      0.74      0.57        73

avg / total       0.93      0.90      0.62      0.91      0.74      0.57        78



### Balance Random Forest Classifier

In [371]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
random_forest = random_forest.fit(X_train, y_train)

In [372]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8178082191780822

In [373]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 4,  1],
       [12, 61]])

In [374]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.25      0.80      0.84      0.38      0.82      0.67         5
          1       0.98      0.84      0.80      0.90      0.82      0.67        73

avg / total       0.94      0.83      0.80      0.87      0.82      0.67        78



### Easy Ensemble AdaBoost Classifier

In [375]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easy = EasyEnsembleClassifier(n_estimators=100, random_state=1)
easy = easy.fit(X_train, y_train)

AttributeError: ignored

In [376]:
# Calculated the balanced accuracy score
y_pred = easy.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

AttributeError: ignored

In [377]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 4,  1],
       [12, 61]])

In [378]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.25      0.80      0.84      0.38      0.82      0.67         5
          1       0.98      0.84      0.80      0.90      0.82      0.67        73

avg / total       0.94      0.83      0.80      0.87      0.82      0.67        78

